<a href="https://colab.research.google.com/github/sanchit2843/Attentiondetection/blob/master/openface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google_drive_downloader import GoogleDriveDownloader as gdd
gdd.download_file_from_google_drive(file_id='1AS0HdPFysd4kTne-taXWPkuDP7qR4_Hn',
                                    dest_path='./data.zip',
                                    unzip=True)

Unzipping...Done.


In [0]:
columns = [' gaze_0_x', ' gaze_0_y', ' gaze_0_z', ' gaze_1_x', ' gaze_1_y',' gaze_1_z'
      , ' gaze_angle_x', ' gaze_angle_y',' pose_Tx', ' pose_Ty', ' pose_Tz', ' pose_Rx'
      , ' pose_Ry', ' pose_Rz',' p_scale', ' p_rx', ' p_ry', ' p_rz', ' p_tx', ' p_ty'
      , ' p_0', ' p_1',' p_2', ' p_3', ' p_4', ' p_5', ' p_6', ' p_7', ' p_8', ' p_9'
      , ' p_10',' p_11', ' p_12', ' p_13', ' p_14', ' p_15', ' p_16', ' p_17', 
      ' p_18', ' p_19', ' p_20', ' p_21', ' p_22', ' p_23', ' p_24', ' p_25', ' p_26',
       ' p_27', ' p_28', ' p_29', ' p_30', ' p_31', ' p_32', ' p_33',
       ' AU01_r', ' AU02_r', ' AU04_r', ' AU05_r', ' AU06_r', ' AU07_r',
       ' AU09_r', ' AU10_r', ' AU12_r', ' AU14_r', ' AU15_r', ' AU17_r',
       ' AU20_r', ' AU23_r', ' AU25_r', ' AU26_r', ' AU45_r', ' AU01_c',
       ' AU02_c', ' AU04_c', ' AU05_c', ' AU06_c', ' AU07_c', ' AU09_c',
       ' AU10_c', ' AU12_c', ' AU14_c', ' AU15_c', ' AU17_c', ' AU20_c',
       ' AU23_c', ' AU25_c', ' AU26_c', ' AU28_c', ' AU45_c']

In [0]:
import numpy as np
from sklearn.preprocessing import normalize
from tqdm.autonotebook import tqdm
import math
import pandas as pd
import torch
def create_dataset(path,label_file,res):
  os.makedirs(res,exist_ok = True)
  data_complete = {}
  label = pd.read_csv(label_file).values
  for i in tqdm(range(len(label))):
    path_file = os.path.join(path,label[i][0][:6],label[i][0][:-4],label[i][0][:-3] + 'csv')
    if(os.path.exists(path_file)):
      pass
    else:
      continue
    file = path_file.split('/')[-1]
    try:
      data = pd.read_csv(path_file)
    except:
      continue
    if(data.shape[0] == 300):
      data_complete[file] = (normalize(data[columns].values,axis = 1),label[i][2])
    if(i%1000 == 0 and i != 0):
      torch.save(data_complete,os.path.join(res,'train_{}.pt'.format(int(i/100))))
      data_complete = {}
  torch.save(data_complete,os.path.join(res,'train_{}.pt'.format(math.ceil(i/100))))
  flag = 0
  for i in os.listdir(res):
    if(os.path.isfile(os.path.join(res,i))):
      if(flag == 0):
        d1 = torch.load(os.path.join(res,i))
        flag = 1
      else:
        d2 = torch.load(os.path.join(res,i))
        d1.update(d2)
  torch.save(d1,res + '.pt')

In [0]:
dir = '/content/Output/Test'
create_dataset(dir,'/content/TestLabels.csv','/content/Test')
dir = '/content/Output/Validation'
create_dataset(dir,'/content/ValidationLabels.csv','/content/Validation')
dir = '/content/Output/Train'
create_dataset(dir,'/content/TrainLabels.csv','/content/Train')

In [0]:
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import os
from tqdm.autonotebook import tqdm
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler
import cv2
import sys

In [0]:
class video_dataset(Dataset):
  def __init__(self,pt):
    self.pt = torch.load(pt)
    self.key = list(self.pt.keys())
  def __len__(self):
    return len(self.key)
  def __getitem__(self,idx):
    k = self.key[idx]
    data,label = self.pt[k]
    return data,label

In [0]:
train_data = video_dataset('/content/Train.pt')
train_loader = DataLoader(train_data,batch_size = 64, num_workers = 4 , shuffle = True)
valid_data = video_dataset('/content/Validation.pt')
valid_loader = DataLoader(valid_data,batch_size = 64 , num_workers = 4 )
test_data = video_dataset('/content/Test.pt')
test_loader = DataLoader(test_data,batch_size = 64, num_workers = 4 )

In [0]:
from torch import nn
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.lstm = nn.GRU(input_dim, hidden_dim, layer_dim, batch_first=True,dropout  = 0.3, bidirectional = True)
        self.fc = nn.Linear(2*hidden_dim, 128)
        self.fc1 = nn.Linear(128,output_dim)
        self.relu = nn.LeakyReLU()
    def forward(self, x):
        self.lstm.flatten_parameters()
        out, (hn, cn) = self.lstm(x)
        out = self.relu(self.fc(out[:,-1,:]))
        out = self.fc1(out)
        return out

In [0]:
model = LSTMModel(89,256,2,4).cuda()
learning_rate = 0.005
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) 

In [0]:
import csv
import os
import cv2
import matplotlib.pyplot as plt
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

class Logger(object):
    def __init__(self, path, header):
        self.log_file = open(path, 'w')
        self.logger = csv.writer(self.log_file, delimiter='\t')
        self.logger.writerow(header)
        self.header = header

    def __del(self):
        self.log_file.close()

    def log(self, values):
        write_values = []
        for col in self.header:
            assert col in values
            write_values.append(values[col])

        self.logger.writerow(write_values)
        self.log_file.flush()


def load_value_file(file_path):
    with open(file_path, 'r') as input_file:
        value = float(input_file.read().rstrip('\n\r'))
    return value


def calculate_accuracy(outputs, targets):
    batch_size = targets.size(0)

    _, pred = outputs.topk(1, 1, True)
    pred = pred.t()
    correct = pred.eq(targets.view(1, -1))
    n_correct_elems = correct.float().sum().item()
    return 100* n_correct_elems / batch_size
def update_lr(optimizer, lr):
    for g in optimizer.param_groups:
        g['lr'] = lr
def update_mom(optimizer, mom):
    for g in optimizer.param_groups:
        g['momentum'] = mom

In [0]:
num_epochs = 20
!mkdir results
arch = 'LSTM'
result_path = os.path.join('./results',arch)
os.makedirs(result_path,exist_ok = True)
train_logger = Logger(
            os.path.join(result_path, 'train{}.log'.format(arch)),
            ['epoch', 'loss', 'acc', 'lr'])
train_batch_logger = Logger(
            os.path.join(result_path, 'train_batch{}.log'.format(arch)),
            ['epoch', 'batch', 'iter', 'loss', 'acc', 'lr'])    
val_logger = Logger(
            os.path.join(result_path, 'val{}.log'.format(arch)), ['epoch', 'loss', 'acc'])

mkdir: cannot create directory ‘results’: File exists


In [10]:
import torch
from torch.autograd import Variable
import time
import os
import sys
import os
criterion = nn.CrossEntropyLoss().to('cuda')
def train_epoch(epoch, num_epochs, data_loader, model, criterion, optimizer, epoch_logger, batch_logger, batch_size , result_path):
    print('Training Epoch {}'.format(epoch))
    model.train()
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    accuracies = AverageMeter()
    start_time = time.time()
    end_time = time.time()
    for i, (inputs, targets) in enumerate(data_loader):
        data_time.update(time.time() - end_time)
        inputs = inputs[:,::2,:]
        if torch.cuda.is_available():
            targets = targets.cuda()
            inputs = inputs.cuda()
        outputs = model(inputs.type(torch.cuda.FloatTensor))
        loss = criterion(outputs, targets)
        acc = calculate_accuracy(outputs, targets)
        losses.update(loss.item(), inputs.size(0))
        accuracies.update(acc, inputs.size(0))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        del inputs,targets,outputs
        batch_time.update(time.time() - end_time)
        end_time = time.time()

        batch_logger.log({
            'epoch': epoch,
            'batch': i + 1,
            'iter': (epoch - 1) * len(data_loader) + (i + 1),
            'loss': losses.val,
            'acc': accuracies.val,
            'lr': optimizer.param_groups[0]['lr']
        })
        sys.stdout.write(
                "\r[Epoch %d/%d] [Batch %d / %d] [Time %.2f %.2f] [Data %.2f %.2f] [Loss: %f, Acc: %.2f%%]"
                % (
                    epoch,
                    num_epochs,
                    i,
                    len(data_loader),
                    batch_time.val,
                    batch_time.avg,
                    data_time.val,
                    data_time.avg,
                    losses.avg,
                    accuracies.avg
                    )
                )
    print('\nEpoch time {} mins'.format((end_time-start_time)/60))
    epoch_logger.log({
        'epoch': epoch,
        'loss': losses.avg,
        'acc': accuracies.avg,
        'lr': optimizer.param_groups[0]['lr']
    })
        
    save_file_path = os.path.join(result_path,'save.pth')
    states = {
        'epoch': epoch + 1,
        'arch': arch,
        'state_dict': model.state_dict()
    }
    torch.save(states, save_file_path)
def val_epoch(epoch, num_epochs,data_loader, model, criterion,  logger , batch_size):
    print('Validation Epoch {}'.format(epoch))
    model.eval()
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    accuracies = AverageMeter()

    end_time = time.time()
    for i, (inputs, targets) in enumerate(data_loader):
        data_time.update(time.time() - end_time)
        if torch.cuda.is_available():
            targets = targets.cuda()
            inputs = inputs.cuda()
        inputs = inputs[:,::2,:]
        inputs = Variable(inputs)
        targets = Variable(targets)
        outputs = model(inputs.type(torch.cuda.FloatTensor))
        loss = criterion(outputs, targets)
        acc = calculate_accuracy(outputs, targets)
        losses.update(loss.item(), inputs.size(0))
        accuracies.update(acc, inputs.size(0))
        batch_time.update(time.time() - end_time)
        end_time = time.time()
        del targets,inputs
        sys.stdout.write(
                "\r[Epoch %d/%d] [Batch %d / %d] [Time %.2f %.2f] [Data %.2f %.2f] [Loss: %f, Acc: %.2f%%]"
                % (
                    epoch,
                    num_epochs,
                    i,
                    len(data_loader),
                    batch_time.val,
                    batch_time.avg,
                    data_time.val,
                    data_time.avg,
                    losses.avg,
                    accuracies.avg
                    )
                )
    print('')
    logger.log({'epoch': epoch, 'loss': losses.avg, 'acc': accuracies.avg})
    return losses.avg
def test(model, data_loader ,criterion, batch_size, logger = None):
    print('Testing')
    model.eval()
    losses = AverageMeter()
    accuracies = AverageMeter()
    for i, (inputs, targets) in enumerate(data_loader):
        if torch.cuda.is_available():
            targets = targets.cuda()
            inputs = inputs.cuda()
        inputs = Variable(inputs)
        inputs = inputs[:,::2,:]
        targets = Variable(targets)
        outputs = model(inputs.type(torch.cuda.FloatTensor))
        loss = criterion(outputs, targets)
        acc = calculate_accuracy(outputs, targets)
        losses.update(loss.item(), inputs.size(0))
        accuracies.update(acc, inputs.size(0))
        sys.stdout.write(
                "\r[Batch %d / %d]  [Loss: %f, Acc: %.2f%%]"
                % (
                    i,
                    len(data_loader),
                    losses.avg,
                    accuracies.avg
                    )
                )
    if(logger):
        logger.log({'epoch': epoch, 'loss': losses.avg, 'acc': accuracies.avg})
    print('\nAccuracy {}'.format(accuracies.avg))
    return losses.avg,accuracies.avg
for epoch in range(1,num_epochs):
    train_epoch(epoch, num_epochs,train_loader, model, criterion, optimizer, train_logger, train_batch_logger , 64 , result_path)
    val_epoch(epoch, num_epochs, valid_loader, model, criterion, val_logger , 64)
    test(model, test_loader, criterion , 64)

Training Epoch 1


ValueError: ignored